In [1]:
import cv2
import numpy as np
from collections import deque

In [3]:
# frame->blur->hsv->mask+(erosion+dilation)->contours
#cnts(find the bigest contur)->minEnclosingCircle,moment(centroid)->draw the cicle centroid and outer circle->store the centroid in deque
#draw lines from points of deque(where the 2 consecutive pts are not empty), with the thickness decreasing
capt=cv2.VideoCapture(0)
greenLower=np.array([29, 86, 6])
greenUpper=np.array([64, 255, 255])

pts=deque(maxlen=64) #a deque for contrail making

while capt.isOpened():
    _,frame=capt.read()
    blur=cv2.GaussianBlur(frame,(11,11),0)
    hsv=cv2.cvtColor(blur,cv2.COLOR_BGR2HSV)
    
    mask=cv2.inRange(hsv,greenLower,greenUpper)
    mask1=cv2.erode(mask,None,iterations=3)
    mask2=cv2.dilate(mask1,None,iterations=3)
    
    contours,_=cv2.findContours(mask2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    center=None  #this needs to be declared since it will be later accessed outside the block(where is it been updated)
    
    if len(contours)>0:
        c=max(contours,key=cv2.contourArea)     #outer most contour
        ((x,y),radius)=cv2.minEnclosingCircle(c)   #find the smallest enclosing circle 
        
        M=cv2.moments(c) #helps in finding the centroid
        center=int(M['m10']/M['m00']),int(M['m01']/M['m00'])  #center value updates
        
        if(radius>10):
            cv2.circle(frame,(int(x),int(y)),int(radius),(0,255,255),2)
            cv2.circle(frame,center,5,(0,0,255),-1)
            
    pts.appendleft(center)    #center value is been accessed
    
    for i in range(1,len(pts)):
        if pts[i-1] is None or pts[i] is None:
            continue;
        thickness=int(np.sqrt(64.0/float(i+1))*2.5)  #64 is the length of the deque
        cv2.line(frame,pts[i-1],pts[i],(0,0,255),thickness)
        
    cv2.imshow('mask',mask)
    cv2.imshow('feed',frame)
    
    if(cv2.waitKey(1)==ord('q')):
        capt.release()
        cv2.destroyAllWindows()
        break;

In [6]:
#highlight the object by adjusting the HSV values and simultaneously see the effect of the final output 

cap=cv2.VideoCapture(0)

def nothing(x):
    return None

window=cv2.namedWindow('track')

l_h=cv2.createTrackbar('l_h','track',84,255,nothing)   #dark blue cap
l_s=cv2.createTrackbar('l_s','track',93,255,nothing)
l_v=cv2.createTrackbar('l_v','track',65,255,nothing)
u_h=cv2.createTrackbar('u_h','track',176,255,nothing)
u_s=cv2.createTrackbar('u_s','track',255,255,nothing)
u_v=cv2.createTrackbar('u_v','track',255,255,nothing)

centers=deque(maxlen=32)  #contrail maintanance   # must be declared outside the while loop only. Otherwise no contrail would be made.and length always remains 1

while cap.isOpened():
    ret,frame=cap.read()
    
    cv2.GaussianBlur(frame,(11,11),0)
    
    l_h=cv2.getTrackbarPos('l_h','track')
    l_s=cv2.getTrackbarPos('l_s','track')
    l_v=cv2.getTrackbarPos('l_v','track')
    u_h=cv2.getTrackbarPos('u_h','track')
    u_s=cv2.getTrackbarPos('u_s','track')
    u_v=cv2.getTrackbarPos('u_v','track')
    
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    l_b=np.array([l_h,l_s,l_v])
    u_b=np.array([u_h,u_s,u_v])
    
    mask=cv2.inRange(hsv,l_b,u_b)
    mask=cv2.erode(mask,None,iterations=3)
    mask=cv2.dilate(mask,None,iterations=3)
    
    contours,hierarchy=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    
    if len(contours)>0: 
        max_cont=max(contours,key=cv2.contourArea)
        cv2.drawContours(frame,[max_cont],-1,(255,0,205),2,cv2.LINE_4)
        
        #center tracking and making contrail...
            #center tracking...
        M=cv2.moments(max_cont)
        center=int(M['m10']/M['m00']),int(M['m01']/M['m00'])
        cv2.circle(frame,center,5,(0,255,100),-1)
        
        #for printing shapes....[using the max_cont]
        approx=cv2.approxPolyDP(max_cont,0.01*cv2.arcLength(max_cont,closed=True),closed=True)
        x=approx.ravel()[0]
        y=approx.ravel()[1]
        if len(approx)==4:
            x1,y1,w,h=cv2.boundingRect(approx)
            ratio=float(w)/h
            if ratio>0.95 and ratio<1.05:
                cv2.putText(frame,'square',(x+5,y-10),cv2.FONT_HERSHEY_DUPLEX,1,(0,255,100),2)
            else: 
                cv2.putText(frame,'rectangle',(x+5,y-10),cv2.FONT_HERSHEY_DUPLEX,1,(0,255,100),2)
        else:
            cv2.putText(frame,'circle',(x+5,y-10),cv2.FONT_HERSHEY_DUPLEX,1,(0,255,100),2)
        #ended printing shapes here
    
    centers.appendleft(center)  #works better outside the if block
    
    for i in range(1,len(centers)):   #must be outside the if block. otherwise wont run sometimes.
        if centers[i-1] is None or centers[i] is None:
            continue
        thickness=int(np.sqrt(32.0/float(i+1))*2.5)
        cv2.line(frame,centers[i-1],centers[i],(0,255,100),thickness)
    
    cv2.imshow('HSV',mask)
    cv2.imshow('LIVE',frame)
    if(cv2.waitKey(1)==ord('q')):
        cap.release();
        break;
cv2.destroyAllWindows()